# Using APIs / Data Structures

Using the Dark Sky Forecast API at https://developer.forecast.io, generate a sentence that describes the weather that day:  

Right now it is `TEMPERATURE` degrees out and `SUMMARY`. Today will be `TEMP_FEELING` with a high of `HIGH_TEMP` and a low of `LOW_TEMP`. `RAIN_WARNING`.  

 - `TEMPERATURE` is the current temperature  
 - `SUMMARY` is what it currently looks like (partly cloudy, etc).  
 - `TEMP_FEELING` is whether it will be hot, warm, cold, or moderate. You will probably use `HIGH_TEMP` and your own thoughts and feelings to determine this.  
 - `HIGH_TEMP` is the high temperature for the day.  
 - `LOW_TEMP` is the low temperature for the day.  
 - `RAIN_WARNING` is something like "bring your umbrella!" if it is going to rain at some point during the day.  

In [1]:
import requests

In [2]:
# Getting New York's weather
response = requests.get('https://api.darksky.net/forecast/MY_API_KEY/40.7128,-74.0060')
data = response.json()

# Getting the needed values
temperature = data['currently']['temperature']
summary = data['currently']['summary']

    # I'm using the average of apparent temperature to determine the value of TEMP_FEELING
days = data['daily']['data']
today = days[0]
avg_apparent = (today['apparentTemperatureHigh'] + today['apparentTemperatureLow']) / 2
if avg_apparent <= 55:
    temp_feeling = 'cold'
elif avg_apparent <= 65:
    temp_feeling = 'cool'
elif avg_apparent <= 75:
    temp_feeling = 'mild'
elif avg_apparent <= 85:
    temp_feeling = 'warm'
else:
    temp_feeling = 'hot'

high_temp = today['temperatureHigh']
low_temp = today['temperatureLow']

rain_chance = today['precipProbability']
    # Ref: https://www.thoughtco.com/chance-of-rain-3444366
if rain_chance < 0.2:
    rain_warning = 'It\'s not gonna rain.'
elif rain_chance < 0.3:
    rain_warning = 'There\'s only a slight chance of raining today, an umbrella would be unnecessary.'
elif rain_chance < 0.5:
    rain_warning = 'There\'s a chance of raining today, bring an umbrella if you feel like putting on extra weight to your backpack.'
else:
    rain_warning = 'Bring an umbrella!'

message = 'Right now it is {} degrees out and {}. Today will be {} with a high of {} and a low of {}. {}'.format(temperature, summary.lower(), temp_feeling, high_temp, low_temp, rain_warning)
print(message)

Right now it is 77.23 degrees out and mostly cloudy. Today will be hot with a high of 89.59 and a low of 74.46. It's not gonna rain.


Once you've accomplished that part, use Mailgun to send yourself an email every morning at 8AM telling you the sentence. The subject line of the email should be something like "8AM Weather forecast: January 1, 1970."

Dates are obtained by using `datetime`, using http://strftime.org/ as a reference.

**BONUS**: List the forecasted temperatures as the day goes on.

In [3]:
# Testing out mailgun to make sure it works (has no problem sending emails.)
# requests.post(
#         "https://api.mailgun.net/v3/MY_SANDBOX_DOMAIN/messages",
#         auth=("api", "MY_API_KEY"),
#         data={"from": "Edward Hong <mailgun@MY_SANDBOX_DOMAIN>",
#               "to": ["Edward.YSHF@gmail.com"],
#               "subject": "Testing Some Mailgun Awesomness",
#               "text": "Hi there!"})

### putting things together  
(including the **BONUS** part)

In [4]:
import requests
import datetime

response = requests.get('https://api.darksky.net/forecast/MY_API_KEY/40.7128,-74.0060')
data = response.json()
temperature = data['currently']['temperature']
summary = data['currently']['summary']
days = data['daily']['data']
today = days[0]
avg_apparent = (today['apparentTemperatureHigh'] + today['apparentTemperatureLow']) / 2
if avg_apparent <= 55:
    temp_feeling = 'cold'
elif avg_apparent <= 65:
    temp_feeling = 'cool'
elif avg_apparent <= 75:
    temp_feeling = 'mild'
elif avg_apparent <= 85:
    temp_feeling = 'warm'
else:
    temp_feeling = 'hot'
high_temp = today['temperatureHigh']
low_temp = today['temperatureLow']
rain_chance = today['precipProbability']
if rain_chance < 0.2:
    rain_warning = 'It\'s not gonna rain.'
elif rain_chance < 0.3:
    rain_warning = 'There\'s only a slight chance of raining today, an umbrella would be unnecessary.'
elif rain_chance < 0.5:
    rain_warning = 'There\'s a chance of raining today, bring an umbrella if you feel like putting on extra weight to your backpack.'
else:
    rain_warning = 'Bring an umbrella!'
message0 = 'Right now it is {} degrees out and {}. Today will be {} with a high of {} and a low of {}. {}\n\n'.format(temperature, summary.lower(), temp_feeling, high_temp, low_temp, rain_warning)

hourly = data['hourly']['data']
hour_count = 1
message1 = 'The hourly temperature forecast for the next 24 hours is listed below:\n'
for hour in hourly:
    if hour_count > 24:
        break
    weather = str(hour['temperature']) + '°F'
    message1 += 'Forecast for {} in New York: {}.\n'.format(datetime.datetime.fromtimestamp(int(hour['time'])).strftime('%Y-%m-%d %H:%M:%S'), weather)
    hour_count += 1

right_now = datetime.datetime.now()
date_string = right_now.strftime("%b %d, %Y")
requests.post(
        "https://api.mailgun.net/v3/MY_SANDBOX_DOMAIN/messages",
        auth=("api", "MY_API_KEY"),
        data={"from": "Edward Hong <mailgun@MY_SANDBOX_DOMAIN>",
              "to": ["Edward.YSHF@gmail.com"],
              "subject": "8 a.m. Weather Forecast - {}".format(date_string),
              "text": message0+message1})

<Response [200]>